# DATA V2 Projet 4 OpenClassRoom


## Liens vers notebook pour les analyses de base

[Jupiter recherche donnee base](recherche_donnee_base.ipynb)

## Nettoyage des données
### importation des bibliothèques ainsi que des fichiers

In [1]:
# import de pandas
import pandas as pd

In [2]:
#fonction utilisée surtout dans les .map
#fichier source dans le sous-repertoire fonction, fichier fonction_perso.py
from fonction.fonction_perso import *

In [3]:
#import des fichiers csv dans le repertoire du notebook
population = pd.read_csv('donnee/population.csv')
sous_nutrition = pd.read_csv('donnee/sous_nutrition.csv')
aide_alimentaire = pd.read_csv('donnee/aide_alimentaire.csv')
dispo_alimentaire = pd.read_csv('donnee/dispo_alimentaire.csv')
nettoyage_cereale = pd.read_csv('donnee/FAO_obtenir_liste_cereale.csv')

### Nettoyage des Dataframes 

In [4]:
#fonction perso, pour information:
help(remplaceinferieurpar0)

#remplacer les < par 0
sous_nutrition['Valeur'] = sous_nutrition['Valeur'].map(remplaceinferieurpar0)


Help on function remplaceinferieurpar0 in module fonction.fonction_perso:

remplaceinferieurpar0(valeur)
    remplace ce qui commence par < par un 0.0
    
    Parameters : valeur (void)
    
    Returns:
    void or float : valeur or 0.0



In [5]:
#passer toutes les valeurs en float
sous_nutrition['Valeur'] = sous_nutrition['Valeur'].astype(float)

In [6]:
#pour information, fonction perso
print(multi1000.__doc__)

#sous-nutrition Valeur en million d'habitant, je prefere en millier pour avoir la même echelle que population.cvs
sous_nutrition['Valeur']= sous_nutrition['Valeur'].map(multi1000)

retourne une valeur multiplier par 1000
    
    Parameters:
        valeur:float
        
    Returns:
        valeur*1000:float
    


In [7]:
#apres analyse, j'ai remarqué que j'utilisais les données population et sous-nutrition en nombre d'habitant
sous_nutrition['Valeur']= sous_nutrition['Valeur'].map(multi1000)
population['Valeur']= population['Valeur'].map(multi1000)

In [8]:
#pour information, fonction perso
help(changement_annee)

#changement d'année de la colonne Année de sous-nutrition
sous_nutrition['Année'] = sous_nutrition['Année'].map(changement_annee)


Help on function changement_annee in module fonction.fonction_perso:

changement_annee(str_annee)
    retourne xxxx+1 au lieu de 'xxxx-yyyy'
    
    Parameters:
        str_annee:string
        
    Returns:
        str_annee:string



In [9]:
#pays beneficiaire: je le change en Zone je prefere avoir le même nom de colonne
aide_alimentaire= aide_alimentaire.rename(columns={'Pays bénéficiaire':'Zone'})

### Jointure pour créer des DataFrames 

In [10]:

#jointure pour fusionner population et sous-nutrition (question 1,5,6)
join_population_sous_nutrition = pd.merge(population,sous_nutrition,on=['Zone','Année'])

#changement des noms de colonnes.
join_population_sous_nutrition.columns = ('Zone','Année','Population','Sous_nutrition')

#ajout d'un dataframe pour ajouter l'aide alimentaire

#grouper les pays année en faisant une somme de la qté d'aliments
aide_group = aide_alimentaire.groupby(['Zone','Année'])['Valeur'].agg('sum')

#joindre l'aide alimentaire et le reste
join_population_sous_nutrition = pd.merge(join_population_sous_nutrition,aide_group,on=['Zone','Année'],
                                          how='left')

#changement des noms de colonnes.
join_population_sous_nutrition.columns = ('Zone','Année','Population en nbr d habitant'
                                          ,'Sous_nutrition en nbr d habitant'
                                          ,'Aide')


In [11]:
#jointure population et dispo pour les question 2,3,4,7
population2017 = population.loc[(population['Année']==2017)] #filtrage population pour garder que 2017
population2017.columns = ('Zone','Année','Population en nbr d habitant')
join_pop_dispo = pd.merge(dispo_alimentaire,population2017,on=['Zone'],how='inner')

In [12]:
#création d'un dataframe avec les noms des cereales pour question julien
liste_cereale = pd.DataFrame (nettoyage_cereale['Produit'].unique(),columns=['cereale'])


### Création des fichiers CSV pour effectuer les recherches dans le prochain notebook

In [13]:
#création de fichier CSV
join_population_sous_nutrition.to_csv('donnee_analyse/pop_nutri_aide.csv',index=False)
join_pop_dispo.to_csv('donnee_analyse/pop_dispo.csv',index=False)
liste_cereale.to_csv('donnee_analyse/cereale.csv')

### Lien pour voir les analyses demandées

[jupiter Analyse](Analyse.ipynb)